# Nesesary Imports

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["NVIDIA_API_KEY"]=os.getenv('NVIDIA_API_KEY')
os.environ['LANGSMITH_API_KEY']=os.getenv('LANGSMITH_API_KEY')
os.environ['LANGSMITH_TRACING']=os.getenv('LANGSMITH_TRACING')
os.environ['LANGSMITH_PROJECT']=os.getenv('LANGSMITH_PROJECT',"Your-proejct-name")

In [ ]:
from IPython import display
import pandas as pd
import numpy as np


df = pd.read_csv('../langchain_crawl_results.csv')
df.head()

In [4]:
list_of_urls = (df[df['Type']=='page'].URL).tolist()

In [5]:
from langchain.document_loaders import UnstructuredURLLoader

loader = UnstructuredURLLoader(list_of_urls[:10])

In [6]:
docs = loader.load()

In [ ]:
from langchain.document_loaders import WebBaseLoader

web_loader = WebBaseLoader(list_of_urls[:10])

web_docs = web_loader.load()

In [ ]:
display.Markdown(docs[5].page_content)

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

spliter = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=100)
documents = spliter.split_documents(docs)
documents

In [10]:
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
# nvidia/nv-embedcode-7b-v1 ---> Mistralbased code optimized
# nvidia/llama-3.2-nv-embedqa-1b-v2 ---> Llama based QA optimized
embeddings_model = NVIDIAEmbeddings(
    model="nvidia/llama-3.2-nv-embedqa-1b-v2",
    api_key=os.getenv("NVIDIA_API_KEY"),
    truncate="NONE", )

In [11]:
from langchain_community.vectorstores import FAISS
vectore_store = FAISS.from_documents(documents, embeddings_model)

In [ ]:
similar_result = vectore_store.similarity_search("What is the purpose of the langchain")
display.Markdown(similar_result[0].page_content)

In [13]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
prompt= ChatPromptTemplate.from_template(
    """
    Answer the user follwing qestin based on only on the provided contet:

    {context}


    """
)

In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA

llm_client = ChatNVIDIA(
    model="deepseek-ai/deepseek-r1-distill-llama-8b",
    api_key=os.getenv('NVIDIA_API_KEY'),
    temperature=0.6,
    top_p=0.7,
    max_tokens=4096,
)

In [ ]:
document_chain= create_stuff_documents_chain(
    llm_client, prompt)
document_chain

In [ ]:
from langchain.chains.retrieval import create_retrieval_chain
retriver = vectore_store.as_retriever()
retrival_chain = create_retrieval_chain(retriver, document_chain)
retrival_chain

In [ ]:
repsonse = retrival_chain.invoke({"input":"give me a list of All chat models available"})
display.Markdown(repsonse['answer'])

In [ ]:
try:
    for chunk in retrival_chain.stream({
        "input": "What is the langchain project all about?"
    }):
        if 'answer' in chunk:
            print(chunk['answer'], end="", flush=True)
except Exception as e:
    print(f"Error occurred: {str(e)}")

In [20]:
vectore_store.save_local("langchain_faiss_store")